In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *
from amftrack.pipeline.functions.transport_processing.high_mag_videos.loading import load_video_dataset
from amftrack.pipeline.functions.transport_processing.high_mag_videos.add_BC import *

from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)

In [ ]:
videos_folder = "/projects/0/einf914/videos/"

analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder_root = "/projects/0/einf914/analysis_videos/"
directory_targ = '/projects/0/einf914/transport/'


In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 33,
"20230903_Plate310" : 42,
"20230904_Plate310" : 52,
"20230905_Plate310" : 64,
"20230906_Plate310" : 73,
}
indexes = {
"20230810_Plate441" : 29,
"20230811_Plate441" : 41,
"20230812_Plate441" : 47,
"20230813_Plate441" : 60,
}

In [ ]:
plate_id = "441_20230807"
plate_id_video = "20230810_Plate441"
data_obj = load_video_dataset(plate_id_video, videos_folder, analysis_folder, analysis_folder_root)

In [ ]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
folders = folders.sort_values(by="datetime")

In [ ]:
dropbox_address = "/DATA/CocoTransport/"
upl_targ = dropbox_address

for plate_id_video in list(indexes.keys()):
    data_obj = load_video_dataset(plate_id_video, videos_folder, analysis_folder, analysis_folder_root)

    exp = Experiment(directory_targ)
    i = indexes[plate_id_video]
    exp.load(folders.iloc[i : i + 1], suffix="_labeled")
    for t in range(exp.ts):
        exp.load_tile_information(t)
    add_betweenness(exp,t)
    # break
    data_obj.video_objs = sorted(data_obj.video_objs, key = lambda video : video.dataset['video_int'])
    for vid_obj in data_obj.video_objs:
        drop_targ = os.path.relpath(f"/{vid_obj.dataset['tot_path_drop']}", upl_targ)
        db_address = f"{upl_targ}KymoSpeeDExtract/{drop_targ}"
        if check_hasedges(vid_obj):
            edge_data_csv = pd.read_csv(vid_obj.edge_adr)
            mapping = {}
            for edge in vid_obj.edge_objs:
                if "network_begin" in edge.mean_data.keys():
                    edge_begin = edge.mean_data["network_begin"]
                    edge_end = edge.mean_data["network_end"]    # break
                    network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 
                    mapping[edge.edge_name] = network_edge
            for edge in vid_obj.edge_objs:
                if "network_begin" in edge.mean_data.keys():
                    add_attribute(edge_data_csv, edge, lambda edge: edge.width(t), "width_automate", mapping)
                    add_attribute(edge_data_csv, edge, lambda edge: edge.betweeness(t), "betweenness_automate", mapping)
                    add_attribute(edge_data_csv, edge, lambda edge: get_derivative(edge,t,fun), "betweenness_derivative", mapping)
                    
            edge_data_csv.to_csv(vid_obj.edge_adr,index=False)
            drop_targ = os.path.relpath(f"/{vid_obj.dataset['tot_path_drop']}", upl_targ)
            db_address = f"{upl_targ}KymoSpeeDExtract/{drop_targ}"
            source = vid_obj.edge_adr
            target = db_address+"/edges_data.csv"
            print(source,target)
            upload(source,target)
    #         break
    # break
    
    
# edge.time_data["flux_mean"].mean()
    

In [ ]:
dropbox_address = "/DATA/CocoTransport/"
analysis_folder = "/projects/0/einf914/analysis_videos/"
folder_filter = dropbox_address[5:]

img_infos = glob.glob(
    f"{analysis_folder}{folder_filter}/**/video_data.json", recursive=True
)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    add_info = pd.read_json(address, orient="index").T
    vid_anls_frame = pd.concat([vid_anls_frame, add_info], ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
analysis_frame = vid_anls_frame
print(f"Number of videos to be uploaded: {len(analysis_frame)}")
### LARGE VIDEO ANALYSIS

nr_parallel = np.min([len(analysis_frame.index), 1])

# run_parallel_flows(
#     "flux_upload.py",
#     [analysis_folder, 9, 0.95, 0.005, 200, dropbox_address],
#     analysis_frame,
#     nr_parallel,
#     "2:00:00",
#     "flux_upload",
#     node = "staging",
#     cpus = 1,
#     # dependency = "flux_extract.sh",
#     name_job = "flux_upload.sh"
# )
# clear_output(wait=False)

# print(
#     "Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress"
# )

In [ ]:
analysis_frame["analysis_folder"].iloc[0]

In [ ]:
plate_id_video = "20230812_Plate441"

folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = indexes[plate_id_video]
exp.load(folders.iloc[i : i + 1], suffix="_labeled")
for t in range(exp.ts):
    exp.load_tile_information(t)
    add_betweenness(exp,t)

# load_graphs(exp, directory_targ,indexes = [0])



In [ ]:
mpl.rcParams["figure.dpi"] = 50

from random import choice
vid_obj = data_obj.video_objs[25]
vid_obj.plot_speed_arrows(plot_both=True, video_txt_size=30,plot_flux = True,plot_text=False)


In [ ]:

for edge in vid_obj.edge_objs:
    if "network_begin" in edge.mean_data.keys():
        edge_begin = edge.mean_data["network_begin"]
        edge_end = edge.mean_data["network_end"]    # break
        network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 

In [ ]:
# def get_derivative(edge,t,fun):
edge = network_edge
fun = lambda edge: edge.betweeness(t)
def get_derivative(edge_d,t,fun):
    edges_begin = edge_d.begin.edges(t)
    edges_begin.remove(edge_d)
    edges_end = edge_d.end.edges(t)
    edges_end.remove(edge_d)
    weight_begin = np.sum([fun(edge) for edge in edges_begin])
    weight_end = np.sum([fun(edge) for edge in edges_end])
    return(weight_end-weight_begin)

In [ ]:
edges = get_all_edges(exp,t)
np.sum[len(edges)

In [ ]:
edges_network = []
derivatives = []

coverage_tot = []
for index,vid_obj in enumerate(data_obj.video_objs):
    if vid_obj.dataset['mode']=="BF" and check_hasedges(vid_obj):
        for edge in vid_obj.edge_objs:
            if "network_begin" in edge.mean_data.keys():
                edge_begin = edge.mean_data["network_begin"]
                edge_end = edge.mean_data["network_end"]    # break
                network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 
                if network_edge.is_in(t): 
                    edges_network.append(network_edge)
                    derivatives.append(get_derivative(network_edge,t,fun))

In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *
downsizing = 5
ax,f = plot_full_video( 
    exp,
    t,
    downsizing=5,
    # points=positions_list,
    video_num=[0],
    edges=edges_network,
    dilation=downsizing,
    # region = region,
    figsize = (18,12),
)
for derivative,edge in zip(derivatives,edges_network):
    pixels = edge.pixel_list(t)
    center_arrow = f(pixels[len(pixels)//2])

    begin_arrow = f(pixels[10])
    end_arrow = f(pixels[-10])
    # begin_arrow = edges_network[0].begin.pos(t)
    # end_arrow = edges_network[0].pixel_list(t)[10]
    vector = (end_arrow-begin_arrow)
    vector = vector/np.linalg.norm(vector)*100/downsizing*(1-2*(derivative>0))
    ax.arrow(center_arrow[1],center_arrow[0],vector[1],vector[0],width = 10)
plt.savefig(f"test_{plate_id_video}_derivative.png", dpi=600)
